# Step 2: Categorisation of securities

Resources
structures P:\Working Folders\Hilton\W\structures.xlsm

Reg28 sheet format and formula P:\Working Folders\Hilton\W\Reg28 Checker Format and Formulae.docx

input file P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx

issuer results file P:\Working Folders\Hilton\W\issuers_1_done.xlsx

merged file P:\Working Folders\Hilton\W\merged.xlsx

output file P:\Working Folders\Hilton\W\issuers_2_done.xlsx

#### Resources
 
 'else' in a list comprehension<br>
 https://stackoverflow.com/questions/2951701/is-it-possible-to-use-else-in-a-list-comprehension
 
convert string to variable<br>
https://www.codeproject.com/Questions/1222606/Python-convert-string-and-variable-to-list-name

Apply function to each element of a list<br>
https://stackoverflow.com/questions/25082410/apply-function-to-each-element-of-a-list

Replace Vlookup in Python using Pandas<br>
https://www.youtube.com/watch?v=AHS925L8JVk&t=9s

In [1]:
# create a time difference function
import datetime

def timediff(start, end, decimals = 1):
    if   int((end - start) / 3600) > 0: # non-zero hours
        return str(int(  (end - start) / 3600))           + 'hr '  + \
               str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    elif int((end - start) /   60) > 0: # non-zero hours and minutes
        return str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    else:
        return str(round((end - start) %   60, decimals)) + 'sec'

In [2]:
# libraries, libraries!

    # time, to determine run durations
import time
start_time = time.time()
start_time_overlord = time.time()

    # general libraries
import pandas   as     pd        # for dataframes
import numpy    as     np        # for np.NaN
import re                        # for regex
from   re       import search    # for regex
from   datetime import datetime  # for script run durations

    # utilise openpyxl tools to add excel features to results sheet
import openpyxl        as     px                                          # for adding sort filters to the excel sheet
from   openpyxl.cell   import Cell                                        # to format cells
from   openpyxl.styles import Alignment, Color, PatternFill, Font, Border # to format cells

    # access Excel application to open results sheet
import  win32com.client as win32                                          # library to convert xls to xlsx
excel = win32.gencache.EnsureDispatch('Excel.Application')                # to open excel application

print(timediff(start_time, time.time()))

0.6sec


In [3]:
# get the overridden securities

structures = r'P:\Working Folders\Hilton\W\structures.xlsm'
overrides  = pd.read_excel(structures, sheet_name = 'overrides')  # security data overrides

In [4]:
# set paths
start_time = time.time()

pthCompliance    = r'P:\Investment Operations\GRC\Compliance'            # path to PIM Compliance folder
pthW             = r'P:\Working Folders\Hilton\W'
pth_infra        = pthCompliance + r'\Regulations\Reg 28 Infrastructure' # for gov guarantee (of infrastructure issuers)
results_filename = r'\issuers_1_done.xlsx'                               # stage 1 output file name

print(timediff(start_time, time.time()))

0.0sec


In [5]:
# utility function to check if a string contains a list element
# https://www.geeksforgeeks.org/python-test-if-string-contains-element-from-list/

def res(t_list, t_string):
    return bool([ele for ele in t_list if(ele in t_string)])

In [6]:
# read in a Reg 28 file from Eagle as a dataframe called 'results' to use as input
start_time = time.time()

results_columns = ['Entity Name', 'Investment Type', 'i Issue Name', 'Primary Asset ID', 'CCY', 'Reg28 Classification',
                    'End Market Value', 'Percentage of Market Value', 'Closing Exposure PA', 'CLN', 'FRN', 'ILB', 'BESA', 
                    'repo', 'margin', 'Date', 'MedCirc062022', 'MedCirc122023', 'GovGuar', 'Term', 'Issuer']

results         = pd.read_excel(pthW + results_filename, sheet_name = 'results', usecols = results_columns)
                  #r'P:\Working Folders\Hilton\W\Reg 28 Report - results.xlsx'

print(timediff(start_time, time.time()))

0.1sec


In [7]:
# import the Reg 28 file with all issuer codes assigned a named issuer
structures  = pthW + r'\structures.xlsm'

# specify columns to include in 'reg' dataframe
reg_columns = ['Issuer Name', 'vs structures', 'MCap', 'Ticker', 'Deb', 'Exchange', 'Foreign', 'GPL', 'Fund', 'Bank',
               'Debt', 'Equity', 'Property', 'Commodity', 'Partg_emplr', 'HL', 'HF', 'PEF', 'Other', 'Infra', 'DI900']

reg         = pd.read_excel(structures, sheet_name = 'reg', usecols = reg_columns)
              #r'P:\Working Folders\Hilton\W\structures.xlsm'

In [8]:
# with all instruments assigned an issuer, merge with the data file in order to look up instrument attributes
merged = results.merge(reg,
                      left_on  = 'Issuer',
                      right_on = 'Issuer Name',
                      how      = 'left') # vlookup using pandas https://www.youtube.com/watch?v=AHS925L8JVk&t=9sb

# save the merged workbook here P:\Working Folders\Hilton\W\merged.xlsx
merged.to_excel(pthW + r'\merged.xlsx', sheet_name = 'merged', index = False)

In [1]:
# Equities without market caps # https://www.geeksforgeeks.org/filter-pandas-dataframe-with-multiple-conditions/
eq_no_MCap  = merged.loc[(merged['MCap'].isnull().values) & (merged['Investment Type'] =='EQ')
                    ].drop_duplicates(subset='Primary Asset ID', keep = 'first')['Primary Asset ID']

NameError: name 'merged' is not defined

In [10]:
# Securities without market caps # https://www.geeksforgeeks.org/filter-pandas-dataframe-with-multiple-conditions/
no_MCap     = merged.loc[(merged['MCap'].isnull().values)
                    ].drop_duplicates(subset='Primary Asset ID', keep = 'first')['Primary Asset ID']

In [11]:
#proof that the merged datagrame includes a non-zero market cap for the instrument
merged[merged['Primary Asset ID'] == 'SSS']['MCap']

Series([], Name: MCap, dtype: float64)

In [12]:
# function to get asset class of a derivative
def derv_ft_op(row):
    if   row['Investment Type'] == 'OP' and res(['JADM', 'ZAUS', 'CURR', 'ZAR'], row['i Issue Name'].upper()):
        return 'c'
    if   row['Investment Type'] == 'OP' and res(['ALSI', 'UKX', 'SPX', 'HSCEI', 'SPYQ', 'SX5E'], row['i Issue Name']):
        return 'e'
    elif row['Investment Type'] == 'FT' and row['Property'] == 'P':
        return 'p'
    elif row['Investment Type'] == 'DERV' and 'TRS' in row['Primary Asset ID'].upper():
        return 'e'
    elif row['Investment Type'] == 'DERV' and ('CPI' in row['i Issue Name'].upper() or row['Term'] >= 396):
        return 'd'       
    elif row['Investment Type'] == 'DERV':
        return 'c'
    elif row['Investment Type'] == 'FT' and (row['GPL'] == 'G' or res(['BOND', 'CBT', 'NOTE'], row['i Issue Name'].upper())):
        return 'd'
    elif row['Investment Type'] == 'FT':
        return 'e'    

In [13]:
# (3) identify derivative asset classes
start_time = time.time()

merged['dX']  = merged.apply(lambda row: derv_ft_op(row), axis=1)

print(timediff(start_time, time.time()))

0.0sec


In [14]:
#proof that the merged datagrame includes a non-zero market cap for the instrument
merged[merged['Primary Asset ID'] == 'SSS']['MCap']

Series([], Name: MCap, dtype: float64)

In [15]:
# insert overridden instrument codes
for i in range(len(overrides)): #https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/
    instr = overrides['InstrCode'][i]
    
    if merged['Primary Asset ID'].eq(overrides.loc[i, 'InstrCode']).any(): 
    #https://www.statology.org/pandas-check-if-column-contains-string/
    #https://www.geeksforgeeks.org/python-data-types/ they are str, int, float, complex, list, tuple
        
        # test for 'BESA'
        if isinstance(overrides['BESA'][i], str):
            merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'BESA'    ] = overrides['BESA'    ][i]
            # https://datagy.io/pandas-get-row-number/
            # https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
        
        # test for 'Issuer'
        #if isinstance(overrides['Issuer'][i], str):
            #merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'Issuer'  ] = overrides['Issuer'  ][i]
        
        # test for 'Date'        
        if isinstance(overrides['Date'][i], datetime):
            merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'Date'    ] = overrides['Date'    ][i]
        
        # test for 'Term'        
        if isinstance(overrides['Term'][i], float):
            merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'Term'    ] = overrides['Term'    ][i]            
        
        # test for 'MCap'        
        #if isinstance(overrides['MCap'][i], float):
            #merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'MCap'    ] = overrides['MCap'    ][i]

        # test for 'Property'
        #if isinstance(overrides['Property'][i], str):
            #merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'Property'] = overrides['Property'][i]

        # test for 'Government guarantee'
        if isinstance(overrides['GovGuar'][i], str):
            merged.at[merged[merged['Primary Asset ID'] == instr].index[0], 'GovGuar' ] = overrides['GovGuar' ][i]            


In [16]:
#proof that the merged datagrame includes a non-zero market cap for the instrument
merged[merged['Primary Asset ID'] == 'SSS']['MCap']

Series([], Name: MCap, dtype: float64)

In [17]:
# function that creates a new column based on the values of other columns
# Python: Check if String Contains Substring https://stackabuse.com/python-check-if-string-contains-substring/

def classify_Reg28(row):
   
    # 1.1(a) Cash with SARB-registered bank   
    if ((res(['CASH', 'FWD'], row['Investment Type']) or row['dX'] == 'c') and pd.isna(row['margin']) and row['Bank'] == 's') or (row['Investment Type'] == 'SYTH'):
        return '1.1(a)'
    
    # 1.1(b) Money market instruments with SARB registered bank
    elif (row['Investment Type'] == 'ST' or row['Investment Type'] == 'FI') and row['Bank'] == 's':
        return '1.1(b)'  
    
    # 1.1(c) Positive margin account balance with SARB registered bank
    elif res(['ST', 'CASH'], row['Investment Type']) and row['margin'] == 'm' and row['Bank'] == 's':
        return '1.1(c)'
    
    # 1.2(a) Cash with a foreign bank
    elif (res(['ST', 'CASH', 'FWD', 'SYTH'], row['Investment Type']) or row['dX'] == 'c') and row['Bank'] == 'b' or row['Primary Asset ID'] == '0649317':
        return '1.2(a)'
    
    # 1.2(b) Cash with an African bank
    elif res(['ST', 'CASH'], row['Investment Type']) and row['Bank'] == 'a':
        return '1.2(b)'
    
    # 1.2(c) Money market instruments with foreign bank
    elif (row['Investment Type'] == 'ST' or row['dX'] == 'c' or (row['Investment Type'] == 'FI' and row['Term'] < 396)) and row['Bank'] == 'b':
        return '1.2(c)'  
    
    # 2.1(a) SA government debt, ZAR-denominated
    elif (row['Issuer'] == 'Republic of South Africa' or row['GovGuar'] == 1) and row['CCY'] == 'ZAR':
        return '2.1(a)'
        
    # 2.1(b) Foreign government debt and foreign government guaranteed debt
    elif row['Issuer'] != 'Republic of South Africa' and row['GPL'] == 'G':
        return '2.1(b)'  
 
    # 2.1(c)(i) Debt issued by SARB registered bank with MCap > R20bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 's' and row['Term'] >= 396 and row['MCap'] > 20:
        return '2.1(c)(i)'
 
    # 2.1(c)(ii) Debt issued by SARB registered bank with R2bn < MCap < R20bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 's' and row['Term'] >= 396 and row['MCap'] < 20 and row['MCap'] > 2:
        return '2.1(c)(ii)'
 
    # 2.1(c)(iii) Debt issued by SARB registered bank with MCap < R2bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 's' and row['Term'] >= 396 and row['MCap'] < 20 and row['MCap'] < 2:
        return '2.1(c)(iii)'          
 
    # 2.1(c)(iv) Debt issued by SARB registered, unlisted bank
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 's' and row['Term'] >= 396 and (row['MCap'] == 0 or pd.isna(row['MCap'])):
        return '2.1(c)(iv)'     
              
    # 2.2(a) SA government debt, non-ZAR-denominated
    elif row['Issuer'] == 'Republic of South Africa' and row['CCY'] != 'ZAR':
        return '2.2(a)' 
    
    # 2.2(c)(i) Debt issued by foreign bank with MCap > R20bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 'b' and row['Term'] >= 396 and row['MCap'] > 20:
        return '2.2(c)(i)'
 
    # 2.2(c)(ii) Debt issued by foreign bank with R2bn < MCap < R20bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 'b' and row['Term'] >= 396 and row['MCap'] < 20 and row['MCap'] > 2:
        return '2.2(c)(ii)'
 
    # 2.2(c)(iii) Debt issued by foreign bank with MCap < R2bn
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 'b' and row['Term'] >= 396 and row['MCap'] < 20 and row['MCap'] < 2:
        return '2.2(c)(iii)'          
 
    # 2.2(c)(iv) Debt issued by unlisted foreign bank
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and row['Bank'] == 'b' and row['Term'] >= 396 and pd.isna(row['MCap']):
        return '2.2(c)(iv)'             
          
    # 2.1(d)(i) Listed debt issued by PFMA entities and by listed corporates
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and (row['GPL'] == 'p' or  row['MCap'] > 0) and row['BESA'] == 'B' and row['CCY'] == 'ZAR':
        return '2.1(d)(i)'
          
    # 2.1(d)(ii) Unlisted debt issued by PFMA entities and by listed corporates
    elif (res(['FI', 'ST'], row['Investment Type']) or row['dX'] == 'd') and (row['GPL'] == 'p' or  row['MCap'] > 0) and row['BESA'] != 'B' and row['CCY'] == 'ZAR':
        return '2.1(d)(ii)'
          
    # 2.1(e)(i) Listed debt issued by unlisted corporates
    elif res(['FI', 'ST'], row['Investment Type']) and row['GPL'] != 'p' and  pd.isna(row['MCap']) and row['BESA'] == 'B' and row['CCY'] == 'ZAR':
        return '2.1(e)(i)'
               
    # 2.1(e)(ii) Unlisted debt issued by unlisted corporates
    elif res(['FI', 'ST'], row['Investment Type']) and row['GPL'] != 'p' and  pd.isna(row['MCap']) and row['BESA'] != 'B' and row['CCY'] == 'ZAR':
        return '2.1(e)(ii)'         
          
    # 2.2(d)(i) Listed foreign debt issued by listed corporates
    elif res(['FI', 'ST'], row['Investment Type']) and (row['GPL'] == 'p' or  row['MCap'] > 0) and row['CCY'] != 'ZAR':
        return '2.2(d)(i)'
          
    # 2.2(d)(ii) Unlisted foreign debt issued listed corporates
    elif res(['FI', 'ST'], row['Investment Type']) and (row['GPL'] == 'p' or  row['MCap'] > 0) and row['BESA'] != 'B' and row['CCY'] != 'ZAR':
        return '2.2(d)(ii)'
          
    # 2.2(e)(i) Listed foreign debt issued by unlisted corporates
    elif res(['FI', 'ST'], row['Investment Type']) and row['GPL'] != 'p' and  pd.isna(row['MCap']) and row['BESA'] != 'B' and row['CCY'] != 'ZAR':
        return '2.2(e)(i)'
               
    # 2.2(e)(ii) Unlisted foreign debt issued by unlisted corporates
    elif res(['FI', 'ST'], row['Investment Type']) and row['GPL'] != 'p' and  pd.isna(row['MCap']) and row['BESA'] != 'B' and row['CCY'] != 'ZAR':
        return '2.2(e)(ii)'
          
    # 3.1(a)(i) Listed ordinary and preference shares, market cap >= R20bn
    elif ((row['Investment Type'] == 'EQ' and row['MCap'] >= 20) or row['dX'] == 'e') and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '3.1(a)(i)'
                  
    # 3.1(a)(ii) Listed ordinary and preference shares, market cap >= R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] >= 2 and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '3.1(a)(ii)'
              
    # 3.1(a)(iii) Listed ordinary and preference shares, market cap < R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] < 2 and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '3.1(a)(iii)'
              
    # 3.1(b) Unlisted ordinary and preference shares
    elif row['Investment Type'] == 'EQ' and pd.isna(row['MCap']) and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '3.1(b)'    
          
    # 3.2(a)(i) Listed ordinary and preference shares, market cap >= R20bn
    elif ((row['Investment Type'] == 'EQ' and row['MCap'] >= 20) or row['dX'] == 'e') and row['CCY'] != 'ZAR' and row['Property'] != 'P':
        return '3.2(a)(i)'
                  
    # 3.2(a)(ii) Listed ordinary and preference shares, market cap >= R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] >= 2 and row['CCY'] != 'ZAR' and row['Property'] != 'P':
        return '3.2(a)(ii)'
              
    # 3.2(a)(iii) Listed ordinary and preference shares, market cap < R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] < 2 and row['CCY'] != 'ZAR' and row['Property'] != 'P':
        return '3.2(a)(iii)'
                  
    # 3.2(b) Unlisted ordinary and preference shares
    elif row['Investment Type'] == 'EQ' and pd.isna(row['MCap']) and row['CCY'] != 'ZAR' and row['Property'] != 'P':
        return '3.2(b)'

    # 4.1(a)(i) Listed ordinary and preference shares, market cap >= R20bn
    elif ((row['Investment Type'] == 'EQ' and row['MCap'] >= 10) or row['dX'] == 'p') and row['CCY'] == 'ZAR' and row['Property'] == 'P':
        return '4.1(a)(i)'
                  
    # 4.1(a)(ii) Listed ordinary and preference shares, market cap >= R2bn
    elif row['Investment Type'] == 'EQ'and row['MCap'] >= 3 and row['CCY'] == 'ZAR' and row['Property'] == 'P':
        return '4.1(a)(ii)'
              
    # 4.1(a)(iii) Listed ordinary and preference shares, market cap < R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] < 3 and row['CCY'] == 'ZAR' and row['Property'] == 'P':
        return '4.1(a)(iii)'
              
    # 4.1(b) Unlisted ordinary and preference shares
    elif row['Investment Type'] == 'EQ' and pd.isna(row['MCap']) and row['CCY'] == 'ZAR' and row['Property'] == 'P':
        return '4.1(b)'    
          
    # 4.2(a)(i) Listed ordinary and preference shares, market cap >= R20bn
    elif ((row['Investment Type'] == 'EQ' and row['MCap'] >= 10) or row['dX'] == 'p') and row['CCY'] != 'ZAR' and row['Property'] == 'P':
        return '4.2(a)(i)'
                  
    # 4.2(a)(ii) Listed ordinary and preference shares, market cap >= R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] >= 3 and row['CCY'] != 'ZAR' and row['Property'] == 'P':
        return '4.2(a)(ii)'
              
    # 4.2(a)(iii) Listed ordinary and preference [roperty shares, market cap < R2bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] < 3 and row['CCY'] != 'ZAR' and row['Property'] == 'P':
        return '4.2(a)(iii)'
                  
    # 4.2(b) Unlisted ordinary and preference property shares
    elif row['Investment Type'] == 'EQ' and pd.isna(row['MCap']) and row['CCY'] != 'ZAR' and row['Property'] == 'P':
        return '4.2(b)'  
    
    # 5. Commodities - NewGold, NewPlats and NwPalladium
    #if search(  'NEWGOLD', row['i Issue Name'].upper()) and row['CCY'] == 'ZAR' and row['Primary Asset ID'].find('GLD') != -1:
    elif row['CCY'] == 'ZAR' and not(row['Primary Asset ID'].upper().find('GLD'))   == -1:
        return '5.1(a)(i)'
    elif row['CCY'] == 'ZAR' and not(row['Primary Asset ID'].upper().find('NGPLT')) == -1 or not(row['Primary Asset ID'].upper().find('NGPLD')) == -1:
        return '5.1(a)(ii)'
    
    # 8. Hedge funds
    elif row['HF'] == 'hf': 
        return '8.1(a)(i)' # local fund of hedge funds
    elif row['HF'] == 'fohf':
        return '8.1(a)(ii)' # local hedge funds
    
    # 9. Private equity funds
    elif row['PEF'] == 'pef': 
        return '9.1(a)(i)' # local private equity fund
    elif row['PEF'] == 'fopef':
        return '9.1(a)(ii)' # local fund of private equity funds  
    
    #10. 'Other' - purposely no script here
    
    #11. Infrastructure
    elif row['Infra'] == 'i' and row['GovGuar'] != 1: 
        return '11(b)'
      
    # uncategorised
    else:
        return '--- r28 ---'  

In [18]:
# function that classifies securities in terms of Rergulation 30 based on the values of other columns
# Python: Check if String Contains Substring https://stackabuse.com/python-check-if-string-contains-substring/

def classify_Reg30(row):
    
    # Take Council for Medical Schemes classification as the default
    if pd.notna(row['MedCirc122023']) and pd.isna(row['CLN']):
        return  row['MedCirc122023']
            
    # 7(a)(ii) "Other" securities
    elif res(['FWD', 'FT', 'OP', 'DERV'], row['Investment Type']) or row['repo'] == 'RPC' or row['Primary Asset ID'] == 'SAFEX' or row['Primary Asset ID'] == 'VARMARG':
        return '7(a)(ii)'    
   
    # 1(a)(i) Cash with SARB-registered bank with DI900 >= R5bn, including margin account balances
    elif (row['FRN'] == 1 or res(['CASH', 'ST', 'SYTH'], row['Investment Type']) or row['margin'] == 'm') and row['Bank'] == 's' and row['DI900'] >= 5:
        return '1(a)(i)'
    
    # 1(a)(ii)  Cash with SARB-registered bank with DI900 >= 0.1, including margin account balances
    elif (row['FRN'] == 1 or res(['CASH', 'ST', 'SYTH'], row['Investment Type']) or row['margin'] == 'm') and row['Bank'] == 's' and row['DI900'] >= .1:
        return '1(a)(ii)'
    
    # 1(a)(iii)  Cash with SARB-registered bank, collateralised with RSA government debt under an ISMA
    elif row['Investment Type'] == 'ST' and res(['ISMA', 'Collateralised'], row['Primary Asset ID']):
        return '1(a)(iii)'
    
    # 1(b) Cash with a foreign bank
    elif (row['FRN'] == 1 or res(['CASH', 'ST', 'SYTH'], row['Investment Type']) or row['dX'] == 'c') and (row['Bank'] == 'a' or row['Bank'] == 'b') or row['Primary Asset ID'] == '0649317':
        return '1(b)(i)'  
    
    # 2(a)(i) SA government and government-guaranteed debt
    elif (row['Issuer'] == 'Republic of South Africa' or row['GovGuar'] == 1 or row['dX'] == 'c') and row['CCY'] == 'ZAR':
        return '2(a)(i)'
    
    # 2(a)(ii) SA local authority debt
    elif row['CCY'] == 'ZAR' and row['GPL'] == 'la':
        return '2(a)(ii)' 
    
    # 2(a)(iii) DBSA debt
    elif row['Issuer'] == 'Development Bank of Southern Africa':
        return '2(a)(iii)' 
    
    # 2(a)(iv) IDC debt
    elif row['Issuer'] == 'Industrial Development Corporation of South Africa':
        return '2(a)(iv)' 
    
    # 2(a)(v) INCA debt
    elif row['Issuer'] == 'Infrastructure Finance Corporation Limited':
        return '2(a)(v)'
    
    # 2(a)(vi) Land Bank debt
    elif row['Issuer'] == 'Land and Agricultural Development Bank of South Africa':
        return '2(a)(vi)'
    
    # 2(a)(vii) TCTA debt
    elif row['Issuer'] == 'Trans-Caledon Tunnel Authority':
        return '2(a)(vii)'        
    
    # 2(a)(viii) SANRAL debt
    elif row['Issuer'] == 'SA National Roads Agency SOC Ltd':
        return '2(a)(viii)'             
    
    # 2(a)(ix) Eskom debt
    elif row['Issuer'] == 'Eskom Holdings SOC Ltd':
        return '2(a)(ix)'           
    
    # 2(a)(x) Transnet debt
    elif row['Issuer'] == 'Transnet SOC Ltd':
        return '2(a)(x)'           
    
    # 2(a)(xi) Debt issued by SARB-registered bank with DI900 >= R5bn
    elif row['Bank'] == 's' and pd.isna(row['repo']) and row['DI900'] >= 5:
        return '2(a)(xi)'          
    
    # 2(a)(xii) Debt issued by SARB-registered bank with DI900 >= 0.1bn
    elif row['Bank'] == 's' and pd.isna(row['repo']) and row['DI900'] >= .1:
        return '2(a)(xii)'    

    # 2(a)(xiii) Corporate debt listed on BESA and included in OTHI or ALBI
    #elif res(['FI', 'ST'], row['Investment Type']) and row['BESA'] == 'B' and row['CCY'] == 'ZAR' and pd.isna(row['GPL']) and pd.isna(row['Bank']):
        #return '2(a)(xiii)'   
     
    elif res(['FI', 'ST'], row['Investment Type']) and row['CCY'] == 'ZAR' and row['Exchange'] != 'CTSE' and row['Deb'] != 'd' and row['repo'] != 'RPC' and row['Property'] != 'P':
        return '2(a)(xiv)'  
        
    # 2(b)(i) Foreign debt
    elif res(['FI', 'ST'], row['Investment Type']) and row['CCY'] != 'ZAR':
        return '2(b)(i)'
        
    # 3(a)(i) SA property
    elif res(['EQ', 'FI'], row['Investment Type']) and row['Property'] == 'P' and row['CCY'] == 'ZAR':
        return '3(a)(i)'
    
    # 3(b) Foreign property
    elif res(['EQ', 'FI'], row['Investment Type']) and row['Property'] == 'P' and row['CCY'] != 'ZAR' :
        return '3(b)'             
        
    # 4(a)(i) Unlisted equity and unlisted debt in the JSE's 'Development Capital and Venture Capital' (DCVC) sector
    elif row['Investment Type'] == 'EQ' and row['CCY'] == 'ZAR' and row['Property'] != 'P' and row['Exchange'] == 'DCVC':
        return '4(a)(i)'            

    # 4(a)(ii)(i) JSE Listed equity with market cap > R50bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] > 50 and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '4(a)(ii)(i)'                

    # 4(a)(ii)(ii) JSE Listed equity with market cap > R5bn
    elif row['Investment Type'] == 'EQ' and row['MCap'] >= 5 and row['CCY'] == 'ZAR' and row['Property'] != 'P':
        return '4(a)(ii)(ii)'                     

    # 4(a)(ii)(iii) JSE Listed equity with market cap < R5bn
    elif ((row['Investment Type'] == 'EQ' and row['MCap'] < 5) or pd.isna(row['MCap'])) and row['CCY'] == 'ZAR' and row['Property'] != 'P' and row['Deb'] != 'd':
        return '4(a)(ii)(iii)'    

    # 4(a)(iii)(i) JSE Listed ETFs linked to JSE ALSI
    elif row['CCY'] == 'ZAR' and row['Exchange'] == 'XJSE' and row['Fund'] == 'f' and 'ALSI' in row['Issuer Name']:
        return '4(a)(iii)(i)'      

    # 4(a)(iii)(ii) JSE Listed ETFs not linked to JSE ALSI
    elif row['CCY'] == 'ZAR' and row['Fund'] == 'f' and 'ALSI' not in row['Issuer Name']:
        return '4(a)(iii)(ii)'
    
    # 4(a)(iv)(i) (Unlisted) CISes linked to the ALSI
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'f' and 'ALSI' in row['Issuer Name']:
        return '4(a)(iv)(i)'     
    
    # 4(a)(iv)(ii) (Unlisted) CISes linked to the ALSI
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'f' and 'ALSI' not in row['Issuer Name']:
        return '4(a)(iv)(ii)' 
     
    # 4(a)(v)(i) Insurance policies linked to the JSE ALSI
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'p' and 'ALSI' in row['Issuer Name']:
        return '4(a)(v)(i)'   
     
    # 4(a)(v)(ii) Insurance policies linked to the JSE ALSI
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'p' and 'ALSI' not in row['Issuer Name']:
        return '4(a)(v)(ii)'   
     
    # 4(b)
    elif ((row['Investment Type'] == 'EQ' and row['Property'] != 'P')) and row['CCY'] != 'ZAR' :
        return '4(b)' 
     
    # 5(a) SA corporate debentures
    elif  row['CCY'] == 'ZAR' and row['Deb'] == 'd':
        return '5(a)'    
    
    # 5(b) Foreign corporate debentures
    elif row['CCY'] != 'ZAR' and row['Deb'] == 'd':
        return '5(b)'  
     
    # 6(a)(i) SA unlinked insurance policies
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'p':
        return '6(a)(i)'  
     
    # 6(a)(ii) SA Linked insurance policies
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'p':
        return '6(a)(ii)'      

    # 6(b) Foreign linked insurance policies
    elif row['CCY'] != 'ZAR'  and pd.isna(row['Exchange']) and row['Fund'] == 'p':
        return '6(b)'
    
    # 7(a)(i) SA inventories carried at lesser of book value and realisable value
    elif row['CCY'] == 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'i':
        return '7(a)(i)'
    
    # 7(b) Foreign 'other' securities
    elif row['CCY'] != 'ZAR' and pd.isna(row['Exchange']) and row['Fund'] == 'i':
        return '7(b)'   
         
    # uncategorised
    else:
        return '--- r30 ---'  

In [19]:
# (10) assign Reg 28 and reg 30 classifications to each security
start_time = time.time()

merged['Reg 28 Classification'] = merged.apply(lambda row: classify_Reg28(row), axis=1)
merged['Reg 30 Classification'] = merged.apply(lambda row: classify_Reg30(row), axis=1)

print(timediff(start_time, time.time()))

0.0sec


In [20]:
len(merged.drop_duplicates(subset='Primary Asset ID', keep = 'first'))

454

In [21]:
no_MCap       = merged[(merged['MCap'].isnull().values)
                          ].drop_duplicates(subset='Primary Asset ID', keep = 'first')

len(no_MCap)

155

In [22]:
# (15) identify securities with absent classifications
start_time = time.time()

# drop duplicate 'no issuer' rows https://www.interviewqs.com/ddi-code-snippets/drop-duplicate-rows-pandas
no_cat_r28    = merged[(merged['Reg 28 Classification'] == '--- r28 ---') | (merged['Reg 28 Classification'].isnull())
                   ].drop_duplicates(subset='Primary Asset ID', keep = 'first')
no_cat_r30    = merged[(merged['Reg 30 Classification'] == '--- r30 ---') | (merged['Reg 30 Classification'].isnull())
                   ].drop_duplicates(subset='Primary Asset ID', keep = 'first')
no_derv_ft_op = merged[((merged['Investment Type'] == 'DERV') | (merged['Investment Type'] == 'FT') |
                        (merged['Investment Type'] == 'OP')) & pd.isna(merged['dX'])
                      ].drop_duplicates(subset='Primary Asset ID', keep = 'first')
term_neg      = merged[merged['Term'] < 0].drop_duplicates(subset='Primary Asset ID', keep = 'first')
uniques       = merged.drop_duplicates(subset='Primary Asset ID', keep = 'first')

# Securities without market caps # https://www.geeksforgeeks.org/filter-pandas-dataframe-with-multiple-conditions/
no_MCap       = merged[(merged['MCap'].isnull().values)
                      ].drop_duplicates(subset='Primary Asset ID', keep = 'first')

# Equities without market caps # https://www.geeksforgeeks.org/filter-pandas-dataframe-with-multiple-conditions/
eq_no_MCap    = merged[(merged['MCap'].isnull().values) & (merged['Investment Type'] =='EQ')
                          ].drop_duplicates(subset='Primary Asset ID', keep = 'first')

# (16) identifying securities classified as Reg 28 2.1(d)(ii) and Reg 28 2.1(e)(ii)
lius = merged[(merged['Reg 28 Classification'] == '2.1(d)(ii)')
                          ].drop_duplicates(subset='Primary Asset ID', keep = 'first') # listed issuer unlisted security
uius = merged[(merged['Reg 28 Classification'] == '2.1(e)(ii)')
                          ].drop_duplicates(subset='Primary Asset ID', keep = 'first') # unlisted issuer unlisted security

# (17) write the dataframe to review it as a workbook
writer = pd.ExcelWriter(pthW + r'\issuers_2_done.xlsx', engine  = 'xlsxwriter')

merged.to_excel(       writer, index = False,  sheet_name = 'all')                              # categorised securities
uniques.to_excel(      writer, index = False,  sheet_name = f'uniques ({len(uniques)})')        # unique securities
no_MCap.to_excel(      writer, index = False,  sheet_name = f'no MCaps ({len(no_MCap)})')       # unassigned issuers
eq_no_MCap.to_excel(   writer, index = False,  sheet_name = f'no eq MCaps ({len(eq_no_MCap)})') # unassigned issuers
term_neg.to_excel(     writer, index = False,  sheet_name = f'neg term ({len(term_neg)})')      # -ve terms to maturity
no_derv_ft_op.to_excel(writer, index = False,  sheet_name = f'no dX ({len(no_derv_ft_op)})')    # unassigned derivatives
no_cat_r28.to_excel(   writer, index = False,  sheet_name = f'no R28 ({len(no_cat_r28)})')      # Reg 28 not assigned
no_cat_r30.to_excel(   writer, index = False,  sheet_name = f'no R30 ({len(no_cat_r30)})')      # Reg 30 not assigned
lius.to_excel(         writer, index = False,  sheet_name = f'2.1(d)(ii) ({len(lius)})')        # 2(d)(ii) securities
uius.to_excel(         writer, index = False,  sheet_name = f'2.1(e)(ii) ({len(uius)})')        # 2(d)(ii) securities

writer.save()
writer.close()

C:\Users\hilton.netta\AppData\Local\Temp\ipykernel_15556\3765607604.py:43: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\hilton.netta\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [23]:
# print how many securities have been categorised and how many not
print(  'Total securities                             :', len(merged),
      '\nUnique securities                            :', len(uniques),
      '\nAll securities without market capitalisation :', len(no_MCap),          
      '\nNegative term securities                     :', len(term_neg),
      '\nReg28 2.1(d)(ii) securities                  :', len(lius),
      '\nReg28 2.1(e)(ii) securities                  :', len(uius),
      '\nEquities without market capitalisation       :', len(eq_no_MCap),
      '\nUnassigned derivative asset class            :', len(no_derv_ft_op),
      '\nUnique securities not categorised for Reg 28 :', len(no_cat_r28),
      '\nUnique securities not categorised for Reg 30 :', len(no_cat_r30))

print('\n')
print(timediff(start_time, time.time()))
print(timediff(start_time_overlord, time.time()))

# dataframes: example, no_issuer, no_CLN_issuer
# input file  P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx
# structures  P:\Working Folders\Hilton\W\structures.xlsm
# results     P:\Working Folders\Hilton\W\issuer_1_done.xlsx
# merged      P:\Working Folders\Hilton\W\merged.xlsx
# output      P:\Working Folders\Hilton\W\issuers_2_done.xlsx

Total securities                             : 461 
Unique securities                            : 454 
All securities without market capitalisation : 155 
Equities without market capitalisation       : 0 
Negative term securities                     : 5 
Unassigned derivative asset class            : 0 
Reg28 2.1(d)(ii) securities                  : 17 
Reg28 2.1(e)(ii) securities                  : 55 
Unique securities not categorised for Reg 28 : 0 
Unique securities not categorised for Reg 30 : 0


0.4sec
2.6sec
